In [1]:
# import dependencies
from pathlib import Path
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from datetime import datetime
from sqlalchemy.orm import sessionmaker
import csv

file_path = Path("../main_dataset.csv")
df = pd.read_csv("../main_dataset.csv") # for the dataset without preprocessing
preprocessing_df = pd.read_csv("../preprocessing.csv", low_memory=False) # dataset from the test.ipynb file with preprocessing


In [2]:
preprocessing_df = preprocessing_df[preprocessing_df['release_date'].notna() & preprocessing_df['release_date'].str.match(r'\d{4}-\d{2}-\d{2}')]
# 15,173 removed as they do not include a release date, only the year. 


preprocessing_df.insert(0, 'index', range(1, len(preprocessing_df) + 1))
preprocessing_df = preprocessing_df.drop(columns=['Unnamed: 0'])

column_mapping = {
    'C♯/D♭': 'CD',
    'D♯/E♭': 'DE',
    'F♯/G♭': 'FG',
    'G♯/A♭': 'GA',
    'A♯/B♭': 'AB', 
    'instrumentalness_above_0.5': 'instrumentalness_above_5',
    'instrumentalness_below_0.5': 'instrumentalness_below_5',
    'speechiness_non-speech': 'speechiness_non_speech'
}

# Rename columns
preprocessing_df.rename(columns=column_mapping, inplace=True)

print(preprocessing_df.columns.tolist())
print(preprocessing_df.shape[0])

['index', 'track_uri', 'name', 'artists_names', 'popularity', 'is_playable', 'release_date', 'artists_uris', 'playlist_uris', 'danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'analysis_url', 'duration_ms', 'artists_popularities', 'artists_genres', 'artists_followers', 'album', 'compilation', 'single', 'release_dayofweek', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'release_month', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December', 'time_since_release', 'pitch_names', 'C', 'CD', 'D', 'DE', 'E', 'F', 'FG', 'G', 'GA', 'A', 'AB', 'B', 'mode_minor', 'mode_major', 'speechiness_mixed', 'speechiness_non_speech', 'speechiness_speech', 'instrumentalness_above_5', 'instrumentalness_below_5', 'is_live', 'is_not_live', 'time_signature_4_4', 'time_signature_other', 'broader_genre_category']
211728


In [3]:
song_attribute = ['"index"', 'is_playable', 'danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 
                  'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'Monday', 'Tuesday', 
                  'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'January', 'February', 'March', 'April', 
                  'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December', 'time_since_release', 
                  'C', 'CD', 'D', 'DE', 'E', 'F', 'FG', 'G', 'GA', 'A', 'AB', 'B', 'mode_minor', 'mode_major', 
                  'speechiness_mixed', 'speechiness_non_speech', 'speechiness_speech', 'instrumentalness_above_5', 
                  'instrumentalness_below_5', 'is_live', 'is_not_live', 'time_signature_4_4', 'time_signature_other']


other_attribute = ['"index"', 'track_uri', 'name', 'artists_names', 'popularity', 'release_date', 
                   'artists_uris', 'playlist_uris', 'artists_popularities', 'artists_genres', 
                   'artists_followers', 'album', 'compilation', 'single', 'release_dayofweek', 
                   'release_month', 'pitch_names', 'broader_genre_category']

In [4]:
# create engine and new sqlite file 
engine = create_engine('sqlite:///converted_file.sqlite')
preprocessing_df.to_sql('dataset', engine, if_exists = 'replace', index=False)

# start session 
Session = sessionmaker(bind = engine)
session = Session()

# print first 10 rows of dataset 
result = session.execute('select * from dataset limit 5')
for row in result:
    print(row) 

(1, 'spotify:track:0GYpisgsrUSd8B4UqksmfY', '新緑', "['上野大樹']", 25, 1, '2023-04-05', "['spotify:artist:5YPkOSqagMwjOWf7PLjHNF']", "['spotify:playlist:37i9dQZF1DWSt89CX9de4L']", 0.595, 0.432, 2.0, -9.488, 0.0422, 0.601, 0.0, 0.0899, 0.379, 145.707, 'https://api.spotify.com/v1/audio-analysis/0GYpisgsrUSd8B4UqksmfY', 260033.0, '[48]', "[['j-acoustic', 'j-pop']]", '[69413]', 1, 0, 0, 2.0, 0, 0, 1, 0, 0, 0, 0, 4.0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 421.0, 'D', 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 'Pop')
(2, 'spotify:track:3b9eglykqfxtTaSpSidP9u', 'またね', "['Lucky Kilimanjaro']", 47, 1, '2023-04-05', "['spotify:artist:2V8UZPMR1EbkXhzvEGBTrV']", "['spotify:playlist:37i9dQZF1DXahYFr91pFvG']", 0.807, 0.707, 5.0, -4.759, 0.0319, 0.0696, 0.0033, 0.31, 0.607, 123.023, 'https://api.spotify.com/v1/audio-analysis/3b9eglykqfxtTaSpSidP9u', 216730.0, '[49]', "[['city pop', 'j-indie', 'japanese electropop', 'japanese r&b']]", '[89318]', 0, 0, 1, 2.0, 0, 0, 1, 0, 0, 0, 0, 4.

In [5]:
other_attributes_query = f"select {', '.join(other_attribute)} from dataset"
result = session.execute(other_attributes_query)
for i, row in enumerate(result):
    if i >= 5:
        break
    print(row)

(1, 'spotify:track:0GYpisgsrUSd8B4UqksmfY', '新緑', "['上野大樹']", 25, '2023-04-05', "['spotify:artist:5YPkOSqagMwjOWf7PLjHNF']", "['spotify:playlist:37i9dQZF1DWSt89CX9de4L']", '[48]', "[['j-acoustic', 'j-pop']]", '[69413]', 1, 0, 0, 2.0, 4.0, 'D', 'Pop')
(2, 'spotify:track:3b9eglykqfxtTaSpSidP9u', 'またね', "['Lucky Kilimanjaro']", 47, '2023-04-05', "['spotify:artist:2V8UZPMR1EbkXhzvEGBTrV']", "['spotify:playlist:37i9dQZF1DXahYFr91pFvG']", '[49]', "[['city pop', 'j-indie', 'japanese electropop', 'japanese r&b']]", '[89318]', 0, 0, 1, 2.0, 4.0, 'F', 'R&B/Soul/Funk')
(3, 'spotify:track:2iOn50LwBLQxhL5pMoIMpW', 'Sueño Flamenco - Original Mix', "['Marksman']", 5, '2023-04-04', "['spotify:artist:59ggRM2BowvTe9nlnwGlBT']", "['spotify:playlist:1ILlyqabDcTv0mbnTxE7Od']", '[26]', '[[]]', '[6409]', 0, 0, 1, 1.0, 4.0, 'F♯/G♭', 'Miscellaneous')
(4, 'spotify:track:68aueb4O4xxqwsBPiP7dLS', 'Nightshift', "['Khainz']", 27, '2023-03-31', "['spotify:artist:71yD5VENn9Wy1IECnpYWvX']", "['spotify:playlist:37i9dQZ

In [6]:
# song_attributes_query = f"select {', '.join(song_attribute)} from dataset limit 5"

song_attributes_query = """
select
    "index", is_playable, danceability, energy, key, loudness, speechiness, acousticness, 
    instrumentalness, liveness, valence, tempo, duration_ms, Monday, Tuesday, 
    Wednesday, Thursday, Friday, Saturday, Sunday, January, February, March, April, 
    May, June, July, August, September, October, November, December, time_since_release, 
    C, CD, D, DE, E, F, FG, G, GA, A, AB, B, mode_minor, mode_major, 
    speechiness_mixed, speechiness_non_speech, speechiness_speech, instrumentalness_above_5, 
    instrumentalness_below_5, is_live, is_not_live, time_signature_4_4, time_signature_other
from dataset 
"""

result = session.execute(song_attributes_query)
for i, row in enumerate(result):
    if i >= 5:
        break
    print(row)


(1, 1, 0.595, 0.432, 2.0, -9.488, 0.0422, 0.601, 0.0, 0.0899, 0.379, 145.707, 260033.0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 421.0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0)
(2, 1, 0.807, 0.707, 5.0, -4.759, 0.0319, 0.0696, 0.0033, 0.31, 0.607, 123.023, 216730.0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 421.0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0)
(3, 1, 0.597, 0.814, 6.0, -7.531, 0.0475, 0.000751, 0.857, 0.105, 0.0917, 124.01, 491613.0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 422.0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0)
(4, 1, 0.757, 0.913, 7.0, -7.542, 0.0483, 0.00333, 0.853, 0.133, 0.181, 123.993, 390968.0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 426.0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0)
(5, 1, 0.641, 0.758, 1.0, -5.04, 0.046, 0.102, 0.0, 0.337, 0.933, 97.969, 211747.0, 0, 0, 0, 1,

In [7]:
song_attributes = ['is_playable', 'danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness',
                   'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'Monday', 'Tuesday',
                   'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'January', 'February', 'March', 'April',
                   'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December', 'time_since_release',
                   'C', 'CD', 'D', 'DE', 'E', 'F', 'FG', 'G', 'GA', 'A', 'AB', 'B', 'mode_minor', 'mode_major',
                   'speechiness_mixed', 'speechiness_non_speech', 'speechiness_speech', 'instrumentalness_above_5',
                   'instrumentalness_below_5', 'is_live', 'is_not_live', 'time_signature_4_4', 'time_signature_other']

additional_attributes = ['track_uri', 'name', 'artists_names', 'popularity', 'release_date', 
                         'artists_uris', 'playlist_uris', 'artists_popularities', 'artists_genres', 
                         'artists_followers', 'album', 'compilation', 'single', 'release_dayofweek', 
                         'release_month', 'pitch_names', 'broader_genre_category']

In [8]:
# Define file paths for CSV files
song_attributes_csv = "song_attributes.csv"
additional_attributes_csv = "additional_attributes.csv"

# Execute and write results for song_attributes_query
with open(song_attributes_csv, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["index"] + song_attributes[1:])
    result = session.execute(song_attributes_query)
    for row in result:
        writer.writerow(row)

# Execute and write results for addl_attributes_query
with open(additional_attributes_csv, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["index"] + additional_attributes[1:])
    result = session.execute(other_attributes_query)
    for row in result:
        writer.writerow(row)